# Week 4 – PySpark Transformations & Modeling

## Learning Objectives
By the end of this session, you will:
- 🔄 Apply schema evolution and enforcement with Delta Lake
- 🚀 Build incremental pipelines using MERGE and timestamp-based logic
- 📊 Use joins, aggregations, and window functions for analytical metrics
- 🏆 Create Gold-layer tables for revenue, retention, and customer metrics
- 🤖 Use Databricks AI to define schemas and generate PySpark + SQL DDL
- ✅ Validate business-friendly analytical models in Databricks

## Setup and Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
import pyspark.sql.types as T

## 1. Schema Evolution, Enforcement & Incremental Delta Processing

### Why Schema Evolution Matters
Modern data systems receive changing datasets: new columns, modified types, updated structures.

Delta Lake's schema evolution + enforcement ensures:
- ✅ Reliable ingestion even when schemas shift
- 🛡️ Protection against invalid/incorrect data
- 🔧 Ability to process changing feeds without rebuilding pipelines

### Example: Basic Schema Change Detection

In [ ]:
# Load different versions of orders data
df1 = spark.table("bronze.orders_v1")
df2 = spark.table("bronze.orders_v2")

print("Orders V1 Schema:")
df1.printSchema()

print("\nOrders V2 Schema:")
df2.printSchema()

### Enabling Schema Evolution

In [ ]:
# Enable schema evolution when writing to Delta table
df2.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .saveAsTable("silver.orders")

### Delta Table History

In [ ]:
# Check Delta table history
delta_tbl = DeltaTable.forName(spark, "silver.orders")
delta_tbl.history().show()

## 2. Incremental Data Processing & MERGE Operations

### When to Use Incremental Pipelines
**Use Cases:**
- 🔄 Continuous ingestion
- ⏰ Late-arriving events
- 💼 Updated business transactions
- 📊 Slowly changing dimensions

### Typical Incremental MERGE Pattern

In [ ]:
# Load incremental data
incremental_df = spark.table("bronze.orders_incremental")
delta_tbl = DeltaTable.forName(spark, "silver.orders")

# Perform MERGE operation
delta_tbl.alias("t").merge(
    incremental_df.alias("s"),
    "t.order_id = s.order_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

print("MERGE operation completed successfully!")

## 3. Joins, Aggregations & Window Functions

### Joining Customer and Order Data

In [ ]:
# Load tables
orders = spark.table("silver.orders")
customers = spark.table("silver.customers")

# Join data
joined = orders.join(customers, "customer_id", "left")
joined.show(5)

### Aggregations for Metrics

In [ ]:
# Calculate customer metrics
metrics = orders.groupBy("customer_id").agg(
    sum("amount").alias("total_revenue"),
    count("*").alias("order_count"),
    avg("amount").alias("avg_order_value")
)

metrics.show(10)

### Window Functions
Powerful for retention, running totals, LTV, and ordering.

In [ ]:
# Define window specification
w = Window.partitionBy("customer_id").orderBy("order_date")

# Apply window functions
df = orders.withColumn("prev_order", lag("order_date").over(w)) \
           .withColumn("lifetime_spend", sum("amount").over(w))

df.select("customer_id", "order_date", "amount", "prev_order", "lifetime_spend").show(10)

## 4. Designing Gold-Layer Tables for Analytics

### Why Gold Tables Exist
Gold tables serve business stakeholders:
- 💰 Revenue metrics
- 🔄 Retention indicators
- 📊 Customer activity signals
- 📈 Aggregates and KPIs
- 🎯 Dashboard-ready structures

### Example: Gold Schema Definition (AI-assisted)

In [ ]:
# Create Gold table schema
ddl = """
CREATE TABLE IF NOT EXISTS gold.customer_metrics (
    customer_id STRING,
    total_revenue DOUBLE,
    avg_order_value DOUBLE,
    order_count INT,
    lifetime_value DOUBLE,
    retention_flag BOOLEAN,
    first_order_date DATE,
    latest_order_date DATE
) USING DELTA
"""

spark.sql(ddl)
print("Gold table created successfully!")

## 5. Gold Table Example: Revenue & Retention

In [ ]:
# Create comprehensive gold metrics
gold = joined.groupBy("customer_id").agg(
    sum("amount").alias("total_revenue"),
    avg("amount").alias("avg_order_value"),
    count("*").alias("order_count"),
    min("order_date").alias("first_order_date"),
    max("order_date").alias("latest_order_date")
).withColumn("lifetime_value", col("total_revenue")) \
 .withColumn("retention_flag", col("order_count") > 1)

gold.show(10)

### Writing the Gold Table

In [ ]:
# Write to Gold table
gold.write.mode("overwrite").saveAsTable("gold.customer_metrics")
print("Gold table populated successfully!")

## 6. Incremental Customer Metrics Pipeline

Build a pipeline that:
1. 🔄 Handles schema evolution on multiple versions of an orders feed
2. 🚀 Processes incremental updates using MERGE
3. 📊 Produces customer metrics using joins, aggregations & window functions
4. 🏆 Stores results in a Gold Delta table
5. 🤖 Uses Databricks AI to define schema + generate DDL

### Phase 1 — Schema Evolution 🔄

In [ ]:
# TODO: Load V1, V2, V3 order files
# TODO: Apply schema evolution
# TODO: Inspect table history
# TODO: Document column changes

print("Phase 1 - Schema Evolution: TODO")

### Phase 2 — Incremental MERGE Logic 🚀

In [ ]:
# TODO: Identify incremental key or timestamp
# TODO: Build MERGE logic
# TODO: Validate row counts

print("Phase 2 - Incremental MERGE Logic: TODO")

### Phase 3 — Metrics Development 📊

In [ ]:
# TODO: Join orders + customers
# TODO: Create grouped aggregations
# TODO: Add window-based metrics (LTV, retention)

print("Phase 3 - Metrics Development: TODO")

### Phase 4 — Gold Table 🏆

In [ ]:
# TODO: Use AI to propose schema
# TODO: Generate PySpark + SQL DDL
# TODO: Create and populate Gold table

print("Phase 4 - Gold Table: TODO")

### Phase 5 — Validation ✅

In [ ]:
# TODO: Validate schema
# TODO: Validate row counts
# TODO: Show final records

print("Phase 5 - Validation: TODO")

## 7. Troubleshooting Guide

### Common Issues & Fixes

#### ❌ Schema mismatch
```
Error: Cannot write incompatible data to column
```
**Fix**: Enable mergeSchema or correct types before write.

#### ❌ Many-to-many join explosion
**Fix**: Deduplicate keys or validate join keys before joining.

#### ❌ Window functions fail
**Fix**: Ensure orderBy uses a consistent type (date, timestamp).

#### ❌ MERGE slow
**Fix**: Z-ORDER and OPTIMIZE the table.

In [ ]:
# Example: Optimize Delta table
spark.sql("OPTIMIZE gold.customer_metrics ZORDER BY customer_id")
print("Table optimized successfully!")

## 8. Key Takeaways & Success Criteria

### What You Should Understand
- 🔄 Delta's schema evolution enables flexibility
- 💰 Incremental pipelines reduce compute costs
- 📊 Joins + aggregations + windows create analytical signals
- 🏆 Gold tables provide consistent business metrics
- 🤖 AI speeds up schema design & DDL creation

### Your Notebook Must Include
- ✅ Working schema evolution demo
- ✅ MERGE incremental logic
- ✅ Customer metrics (revenue, order count, LTV)
- ✅ Gold table creation + preview
- ✅ Reflection on learnings

## 9. Reflection & Next Steps

In [ ]:
# Final validation
print("=== Final Pipeline Validation ===")
print(f"Gold table row count: {spark.table('gold.customer_metrics').count()}")
print("\nSample records:")
spark.table('gold.customer_metrics').show(5)

print("\n🎉 Week 4 Pipeline Complete!")